## Imports

In [6]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from urllib.request import urlretrieve
from zipfile import ZipFile
import os

## Dataset Downloading

In [ ]:
DATASET_ZIP_URL = 'https://www.kaggle.com/api/v1/datasets/download/masoudnickparvar/brain-tumor-mri-dataset'
LOCAL_ZIP_FILENAME = 'dataset.zip'

try:
    urlretrieve(DATASET_ZIP_URL, LOCAL_ZIP_FILENAME)
    print(f"ZIP file downloaded successfully to {LOCAL_ZIP_FILENAME}")
except Exception as e:
    print(f"Error downloading file: {e}")
    exit()

In [ ]:
EXTRACT_ZIP_DIR = 'data'

try:
    with ZipFile(LOCAL_ZIP_FILENAME, 'r') as zip_file:
        zip_file.extractall(EXTRACT_ZIP_DIR)
    print(f"Contents extracted successfully to {EXTRACT_ZIP_DIR}")
except Exception as e:
    print(f"Error extracting ZIP file: {e}")